In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [5]:
root_dir = Path().cwd().parent
calib_dir = root_dir / 'data/raw/final_project/calib'
interim_dir = root_dir / 'data/interim'
interim_dir.mkdir(parents=True, exist_ok=True)
undistorted_dir = interim_dir / 'undistorted'
undistorted_dir.mkdir(parents=True, exist_ok=True)

In [10]:
folders = os.listdir(calib_dir)

for folder in folders:
    images = os.listdir(calib_dir / folder / 'data')
    
    objpoints = []
    imgpoints = []
    
    im = cv2.imread(str(calib_dir / folder / 'data' / images[0]))
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    for v,h in zip([5,7,5], [7,11,15]):
        ret = True
        while ret:
            ret, corners = cv2.findChessboardCorners(im_gray, (v,h))
            #if not ret: ret, corners = cv2.findChessboardCornersSB(im_gray, (v,h))
            if not ret: break
            
            imgpoints.append(corners)
            objp = np.zeros((h*v,3), np.float32)
            objp[:,:2] = np.mgrid[0:v,0:h].T.reshape(-1,2)
            objpoints.append(objp)
            
            start_x = int(np.min(corners[:,0].T[0]))
            end_x = int(np.max(corners[:,0].T[0]))
            start_y = int(np.min(corners[:,0].T[1]))
            end_y = int(np.max(corners[:,0].T[1]))
            im_gray = cv2.rectangle(im_gray, (start_x, start_y), (end_x, end_y), 0, -1)

    #cv2.imshow('im', im_gray)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, im_gray.shape[::-1], None, None)

    for image in images:
        im = cv2.imread(str(calib_dir / folder / 'data' / image))
        im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        h,w = im.shape[:2]
        newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
        dst = cv2.undistort(im, mtx, dist, None, newcameramtx)
        dst = dst[roi[1]:roi[1]+roi[3],roi[0]:roi[0]+roi[2]]
        (undistorted_dir / folder).mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(undistorted_dir / folder / os.path.split(image)[-1]), dst)